In [1]:
from jflegDataset import JflegDataset
from torch.utils.data import DataLoader
from _utils import tokenizerSetup, SpecialToken
from model import S2S
import torch

In [2]:

TRAIN_PATH = "dataset/train.csv"
VAL_PATH = "dataset/eval.csv"


In [3]:
tokenizer = tokenizerSetup()

sentences = ["It will rain in the",
            "I want to eat a big bowl of",
            "My dog is"]
a = tokenizer(sentences, return_tensors="pt", padding=True)

a

{'input_ids': tensor([[50259, 50259, 50259,  1026,   481,  6290,   287,   262],
        [   40,   765,   284,  4483,   257,  1263,  9396,   286],
        [50259, 50259, 50259, 50259, 50259,  3666,  3290,   318]]), 'attention_mask': tensor([[0, 0, 0, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 1, 1, 1]])}

In [4]:


ds_train = JflegDataset(TRAIN_PATH, tokenizer)
ds_eval = JflegDataset(VAL_PATH, tokenizer)

dl_train = DataLoader(ds_train, batch_size=2)
dl_eval = DataLoader(ds_eval, batch_size=2)

tokenizer

GPT2Tokenizer(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, is_fast=False, padding_side='left', truncation_side='right', special_tokens={'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '<|endoftext|>', 'pad_token': '[PAD]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	50257: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	50258: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	50259: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
}

In [5]:
it = iter(dl_train)
next(it)
x,y = next(it)
print(x["input_ids"].squeeze())
ds_train.decode(x["input_ids"][0].squeeze())

tensor([[50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
         50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
         50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
         50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
         50259, 50259, 50259, 50259, 50259, 50259, 50259, 50257,   604,    25,
          9930,   423,   257,  1263,  2863,   284,  8335,   329,   511,  2003,
          1204,   220,   220, 50258],
        [50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
         50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
         50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
         50257,   317,  2818,  1672,   286,  6416,  7793,  1134,   373,   618,
           347,  1042,   283,   694, 25036,   262,  3850,  3194,   416,   262,
          5822,   290,  1908,   340,   284,   262,  4141, 23129, 28283,  6711,
           764

' 4:they have a big chance to prepare for their future life  '

In [7]:
model = S2S(tokenizer.vocab_size + len(SpecialToken), 768)

for x,y in dl_train:
    src = x["input_ids"]
    target = y[0]["input_ids"]
    src_mask = x["attention_mask"]
    target_mask = y[0]["attention_mask"]

    print(f"{src.shape=}")
    print(f"{target.shape=}")
    out = model(src,target,src_mask,target_mask)
    print(f"{out.shape=}")
    break

src.shape=torch.Size([2, 64])
target.shape=torch.Size([2, 64])
out.shape=torch.Size([2, 64, 50260])
